# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':')
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
import pandas as pd
import numpy as np
import operator 
document = ET.parse( './data/mondial_database.xml' )

In [6]:
root = document.getroot()
root.tag
root.attrib

{}

In [7]:
cols = ['country', 'infant_mortality']
mat = pd.DataFrame(columns = cols)
for country in root.iterfind('country'):
    if country.find('infant_mortality') is None:
         pass
    else:
        name = country.find('name').text
        infmor = float(country.find('infant_mortality').text)
        mat = mat.append({'country':name, 'infant_mortality':infmor}, ignore_index=True)
        

#ordered_mat = sorted(mat.items(), key=operator.itemgetter(1))

print(mat.head(10))

          country  infant_mortality
0         Albania             13.19
1          Greece              4.78
2       Macedonia              7.90
3          Serbia              6.16
4         Andorra              3.69
5          France              3.31
6           Spain              3.33
7         Austria              4.16
8  Czech Republic              2.63
9         Germany              3.46


In [8]:
ordered_mat = mat.sort_values('infant_mortality')
print(ordered_mat.head(10))

            country  infant_mortality
36           Monaco              1.81
90            Japan              2.13
109         Bermuda              2.48
34           Norway              2.48
98        Singapore              2.53
35           Sweden              2.60
8    Czech Republic              2.63
72        Hong Kong              2.73
73            Macao              3.13
39          Iceland              3.15


In [85]:
data_cities = pd.DataFrame(columns=['City','Population','Year'])

for country in root.iterfind('country'):
    for city in country.getiterator('city'):
        if city.find("population[last()]") is None:
            pass
        else:
            cname = city.find('name').text
            stadt = city.find('./population[last()]')
            cpop = float(city.find("population[last()]").text)
            year = int(stadt.get('year'))
            data_cities = data_cities.append({'City':cname, 'Population':cpop, 'Year':year}, ignore_index=True)

data_cities_sted = data_cities.sort_values('Population', ascending=False)
print(data_cities_sted.head(10))

           City  Population    Year
1251   Shanghai  22315474.0  2010.0
707    Istanbul  13710512.0  2012.0
1421     Mumbai  12442373.0  2011.0
443      Moskva  11979529.0  2013.0
1250    Beijing  11716620.0  2010.0
2594  São Paulo  11152344.0  2010.0
1252    Tianjin  11090314.0  2010.0
974   Guangzhou  11071424.0  2010.0
1467      Delhi  11034555.0  2011.0
977    Shenzhen  10358381.0  2010.0


In [115]:
data_ethpop = pd.DataFrame(columns=['Country','Ethnic Group','Population'])

for country in root.iterfind('country'):
    if country.find('./ethnicgroup[1][@percentage]') is None:
        pass
    else:
        cname = country.find('name').text
        ethname = country.find('./ethnicgroup[1]').text
        ethnic = country.find('./ethnicgroup[1]')
        totpop = float(country.find('population[last()]').text)
        ethper = float(ethnic.get('percentage'))
        ethpop = (totpop * ethper)/100
        data_ethpop = data_ethpop.append({'Country':cname,'Ethnic Group':ethname, 'Population':ethpop}, ignore_index=True)

data_ethpop_sted = data_ethpop.sort_values('Population', ascending=False)
print(data_ethpop_sted.head(10))

           Country Ethnic Group    Population
44           China  Han Chinese  1.245059e+09
55           India    Dravidian  3.027137e+08
97   United States     European  2.549581e+08
165        Nigeria      African  1.626516e+08
53      Bangladesh      Bengali  1.467769e+08
81           Japan     Japanese  1.265342e+08
21          Russia      Russian  1.146462e+08
73       Indonesia     Javanese  1.134560e+08
143         Brazil     European  1.088867e+08
62         Vietnam    Viet/Kinh  7.607838e+07


In [194]:
data_riv = pd.DataFrame(columns=['Country','RiverName','Dist'])
data_riv_multi = pd.DataFrame(columns=['Country','RiverName','Dist'])

for river in root.iterfind('river'):
    fcountry = river.find('source')
    fcname = fcountry.get('country')
    ecountry = river.find('estuary')
    ecname = ecountry.get('country')
    if fcountry.get('country') == ecountry.get('country'):
        #This river is in one country
        rivname = river.find('name').text
        length = river.find('length')
        data_riv = data_riv.append({'Country':fcname,'RiverName':rivname, 'Dist':length}, ignore_index=True)
    else:
        #This river is in multiple countries
        rivname = river.find('name').text
        tots = fcname + ecname
        length = river.find('length')
        data_riv_multi = data_riv_multi.append({'Country':tots,'RiverName':rivname, 'Dist':length}, ignore_index=True)
print(type(length))
print(data_riv.head(10))
print(data_riv_multi.head(10))

        

<class 'xml.etree.ElementTree.Element'>
  Country            RiverName Dist
0      IS              Thjorsa   []
1      IS  Joekulsa a Fjoellum   []
2       N               Glomma   []
3       N                Lagen   []
4       S            Goetaaelv   []
5       S              Umeaelv   []
6       S              Dalaelv   []
7       S       Vaesterdalaelv   []
8       S        Oesterdalaelv   []
9      SF            Ounasjoki   []
  Country  RiverName Dist
0      NS   Klaraelv   []
1   SFR N  Paatsjoki   []
2     SFR     Vuoksi   []
3     FNL       Maas   []
4      EF    Garonne   []
5     CHF      Rhone   []
6      EP       Tajo   []
7      EP      Douro   []
8    EP E   Guadiana   []
9     CHI     Ticino   []


In [180]:
data_lake = pd.DataFrame(columns=['Country','Lake','Size'])

for lake in root.iterfind('lake'):
    if lake.find('./area') is None:
        pass
    else:
        lakename = lake.find('name').text
        countrycode = lake.get('country')
        lakearea = float(lake.find('area').text)
        #print(lakename, countrycode, lakearea)
        data_lake = data_lake.append({'Country':countrycode,'Lake':lakename,'Size':lakearea}, ignore_index=True)

print(data_lake.head(10))

  Country        Lake    Size
0      SF       Inari  1040.0
1      SF  Oulujaervi   928.0
2      SF   Kallavesi   472.0
3      SF      Saimaa  4370.0
4      SF  Paeijaenne  1118.0
5       N  Mjoesa-See   368.0
6       S    Storuman   173.0
7       S      Siljan   290.0
8       S    Maelaren  1140.0
9       S     Vaenern  5648.0


In [179]:
data_airport = pd.DataFrame(columns=['Country','Airport','Elevation'])

for airport in root.iterfind('airport'):
    if airport.find('./elevation') is None:
        pass
    else:
        airname = airport.findtext('name')
        elev = airport.find('elevation').text
        countname = airport.get('country')
        data_airport = data_airport.append({'Country':countname,'Airport':airname,'Elevation':elev}, ignore_index=True)
print(type(elev))
print(data_airport.head(10))

<class 'str'>
  Country               Airport Elevation
0     AFG                 Herat       977
1     AFG            Kabul Intl      1792
2      AL          Tirana Rinas        38
3      DZ  Cheikh Larbi Tebessi       811
4      DZ         Batna Airport       822
5      DZ               Soummam         6
6      DZ           Tamanrasset      1377
7      DZ                Biskra        88
8      DZ  Mohamed Boudiaf Intl       691
9      DZ     Ain Arnat Airport      1024
